# Merge table from OMIM + gnomAD result

gnomAD로부터 계산한 variants 들을 하나의 파일로 합치며, OMIM 아이디를 기준으로 OMIM에서 제공하는 API를 활용하여 새로운 정보를 추가하는 코드입니다.

https://api.omim.org/api/html

In [1]:
in_xlsx_pattern = 'data/*.recode.picked.header.xlsx'

In [2]:
import glob
import pandas as pd

In [138]:
infiles = glob.glob(in_xlsx_pattern)

In [139]:
dfs=[]
for infile in infiles:
    df = pd.read_excel(infile)
    dfs.append(df)

In [140]:
result = pd.concat(dfs)

In [141]:
MIM_IDs= result['omim'].unique()
MIM_IDs

array([607542, 607840, 600163, 601094, 613113, 602421])

In [142]:
result.head()

,omim,CHROM,POS,ID,REF,ALT,AC,variant_portion,AF,동아시아인,한국인,일본인,한일 외 동아시아인,아프리카인/아프리카계 아메리카인,라틴/혼혈아메리카인,핀란드외 유럽인,"독,폴,러계 유대인",핀란드인,남아시아인,알수없음
0,607542,11,2790111,rs17215500,C,T,26,0.333333,0.000103,NaN,NaN,NaN,NaN,0.000062,0.000087,0.000176,NaN,0.000046,0.000033,NaN
1,607542,11,2799239,rs120074190,G,A,12,0.153846,0.000048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000560,NaN,NaN
2,607542,11,2594193,rs120074187,G,A,12,0.153846,0.000048,NaN,NaN,NaN,NaN,NaN,0.000261,0.000018,0.0001,NaN,NaN,NaN
3,607542,11,2799220,rs17221854,C,T,4,0.051282,0.000016,NaN,NaN,NaN,NaN,NaN,NaN,0.000026,NaN,NaN,0.000033,NaN
4,607542,11,2594209,rs120074186,G,A,4,0.051282,0.000016,NaN,NaN,NaN,NaN,0.000062,0.000058,NaN,NaN,NaN,0.000033,NaN


## OMIM 불러오기

In [8]:
import os, json, requests, time
from pandas.io.json import json_normalize
import urllib
import ast

In [11]:
OMIM_URL        = 'http://api.omim.org/api/entry?'
OMIM_APIKEY     = 'apiKey=[APIKEY_blahblah]&format=python&'
# Constructing query string:
URL = OMIM_URL

for ID in MIM_IDs:
    URL += 'mimNumber=%s&' % ID

# Retrieving the following fields:
URL += 'include=text&'
URL += 'include=allelicVariantList&'
URL += 'include=referenceList&'

# Adding API key:
URL += OMIM_APIKEY


# Downloading page:
page = requests.get(URL)

# Reconstructingh dictionary from the returned string:
OMIM_entry  = ast.literal_eval(page.content)


In [56]:
len(OMIM_entry['omim']['entryList'])

6

In [143]:
OMIM_data={}
# Parsing hash:
for entry in OMIM_entry['omim']['entryList']:
    # GEt OMIM ID:
    ID = entry["entry"]["mimNumber"]
    OMIM_data[ID] = {}

    # Get OMIM name:
    OMIM_data[ID]['title'] = entry["entry"]["titles"]['preferredTitle']

    # Get OMIM text:
    OMIM_data[ID]['text'] = {}
    for fields in entry['entry']["textSectionList"]:
        OMIM_data[ID]['text'][fields['textSection']['textSectionTitle']] = fields['textSection']['textSectionContent']

    # now we have to parse allelic variants:
    # print stuff['omim']['entryList'][0]['entry']['allelicVariantList'][0]['allelicVariant'].keys()
    # ['status', 'name', 'dbSnps', 'text', 'mutations', 'number', 'alternativeNames', 'clinvarAccessions']
    

    OMIM_data[ID]['variations'] = {}
    for variations in entry['entry']['allelicVariantList']:
        if 'dbSnps' in variations['allelicVariant']:
            dbSnps_list = variations['allelicVariant']['dbSnps'].split(',')
#            if ',' in variations['allelicVariant']['dbSnps']:
#                print (dbSnps_list)
#            else:
#                print (dbSnps_list)
            for dbSnps in dbSnps_list:
                OMIM_data[ID]['variations'][dbSnps] = variations['allelicVariant']['mutations']
            #rint(ID, variations['allelicVariant']['dbSnps'], OMIM_data[ID]['variations'][variations['allelicVariant']['dbSnps']])


In [144]:
OMIM_data.keys()

[607840, 600163, 601094, 602421, 607542, 613113]

In [145]:
OMIM_data[602421]["variations"]['rs113993958']

'CFTR, ASP110HIS'

In [146]:
OMIM_data[602421]["variations"]['rs75528968']

'CFTR, 1-BP DEL, 1078T'

In [147]:
OMIM_data[602421]["variations"]['rs121908744']

'CFTR, 1-BP DEL, 1078T'

In [148]:
OMIM_data[602421]["title"]

'CYSTIC FIBROSIS TRANSMEMBRANE CONDUCTANCE REGULATOR; CFTR'

In [149]:
result[['omim','ID']].head(2)

,omim,ID
0,607542,rs17215500
1,607542,rs120074190


In [150]:
result['omim_mut'] = result.apply(lambda x : OMIM_data[x["omim"]]["variations"][x["ID"]] if x["ID"] in OMIM_data[x["omim"]]["variations"] else None, axis=1)

In [151]:
result['omim_title'] = result.apply(lambda x : OMIM_data[x["omim"]]["title"], axis=1)

In [152]:
cols = result.columns.tolist()
cols.insert(0, cols.pop())
cols.insert(0, cols.pop())

In [153]:
result = result.reindex(columns=cols)

In [154]:
result.head()

,omim_mut,omim_title,omim,CHROM,POS,ID,REF,ALT,AC,variant_portion,...,한국인,일본인,한일 외 동아시아인,아프리카인/아프리카계 아메리카인,라틴/혼혈아메리카인,핀란드외 유럽인,"독,폴,러계 유대인",핀란드인,남아시아인,알수없음
0,"KCNQ1, ARG518TER","POTASSIUM CHANNEL, VOLTAGE-GATED, KQT-LIKE SUB...",607542,11,2790111,rs17215500,C,T,26,0.333333,...,NaN,NaN,NaN,0.000062,0.000087,0.000176,NaN,0.000046,0.000033,NaN
1,"KCNQ1, GLY589ASP","POTASSIUM CHANNEL, VOLTAGE-GATED, KQT-LIKE SUB...",607542,11,2799239,rs120074190,G,A,12,0.153846,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000560,NaN,NaN
2,"KCNQ1, ALA300THR","POTASSIUM CHANNEL, VOLTAGE-GATED, KQT-LIKE SUB...",607542,11,2594193,rs120074187,G,A,12,0.153846,...,NaN,NaN,NaN,NaN,0.000261,0.000018,0.0001,NaN,NaN,NaN
3,"KCNQ1, ARG583CYS","POTASSIUM CHANNEL, VOLTAGE-GATED, KQT-LIKE SUB...",607542,11,2799220,rs17221854,C,T,4,0.051282,...,NaN,NaN,NaN,NaN,NaN,0.000026,NaN,NaN,0.000033,NaN
4,"KCNQ1, TRP305SER","POTASSIUM CHANNEL, VOLTAGE-GATED, KQT-LIKE SUB...",607542,11,2594209,rs120074186,G,A,4,0.051282,...,NaN,NaN,NaN,0.000062,0.000058,NaN,NaN,NaN,0.000033,NaN


In [155]:
result.to_excel("omim_merged.xlsx", index=False)